[Reference](https://medium.com/@Rohan_Dutt/how-to-build-mlops-pipelines-that-detect-data-drift-before-it-becomes-a-business-problem-28dceeb8d7cb)

# Detecting Drift Before It Becomes a Crisis

In [1]:
import pandas as pd
from scipy.stats import ks_2samp

# Simulated training data distribution
train_data = pd.Series([0.1, 0.5, 0.7, 0.2, 0.9])
# Real-time incoming data
new_data = pd.Series([0.15, 0.55, 0.65, 0.25, 0.95])
# Kolmogorov-Smirnov test to detect distribution drift
stat, p_value = ks_2samp(train_data, new_data)
if p_value < 0.05:
    print("⚠️ Data drift detected! Investigate immediately.")
else:
    print("Data distribution within expected range ✅")

Data distribution within expected range ✅


/usr/local/lib/python3.12/dist-packages/scipy/stats/_axis_nan_policy.py:579: RuntimeWarning: ks_2samp: Exact calculation unsuccessful. Switching to method=asymp.
  res = hypotest_fun_out(*samples, **kwds)


# Monitoring With Context

In [2]:
def monitor_feature_drift(feature_name, baseline, current):
    stat, p = ks_2samp(baseline, current)
    if p < 0.05:
        return f"⚠️ Drift in {feature_name}: investigate!"
    return f"{feature_name} OK ✅"

# Example
print(monitor_feature_drift("premium_user_spend", train_data, new_data))

premium_user_spend OK ✅


# Python in Action: Rolling Window Drift Detection


In [3]:
import pandas as pd
from scipy.stats import ks_2samp

# Simulated historical and recent feature data
historical = pd.Series([100, 102, 98, 101, 99])
recent = pd.Series([105, 108, 103, 107, 110])
# Rolling window KS test
stat, p_value = ks_2samp(historical, recent)
if p_value < 0.05:
    print("⚠️ Drift detected! Consider retraining.")
else:
    print("Feature distribution stable ✅")

⚠️ Drift detected! Consider retraining.


# Dynamic Threshold Alerts for Business Impact

In [4]:
# Example: alert if revenue-related feature drifts more than 5%
threshold = 0.05
drift_pct = abs(recent.mean() - historical.mean()) / historical.mean()

if drift_pct > threshold:
    print(f"⚠️ Revenue feature drifted {drift_pct:.2%}! Trigger retrain.")
else:
    print("Revenue feature within safe range ✅")

⚠️ Revenue feature drifted 6.60%! Trigger retrain.


# Python in Action: Detecting Missing Data Drift


In [5]:
import pandas as pd

# Historical feature distribution
historical = pd.Series([1, 0, 1, 1, 0, 1])  # 1 = payment_status filled, 0 = null
# Incoming production data
current = pd.Series([1, 0, 0, 0, 0, 1])
# Calculate null proportion drift
historical_null_rate = (historical == 0).mean()
current_null_rate = (current == 0).mean()
drift_threshold = 0.2  # alert if null rate increases by 20%
if current_null_rate - historical_null_rate > drift_threshold:
    print(f"⚠️ Missing data drift detected! Null rate jumped from {historical_null_rate:.0%} to {current_null_rate:.0%}")
else:
    print("Data null rate within expected range ✅")

⚠️ Missing data drift detected! Null rate jumped from 33% to 67%


# Python in Action: Business-Weighted Drift Alerts


In [6]:
import pandas as pd

# Simulated feature drift scores and business impact
alerts = pd.DataFrame({
    "feature": ["payment_status", "user_age", "transaction_amount", "promo_code"],
    "drift_score": [0.35, 0.05, 0.25, 0.02],  # e.g., KS test p-values inverted
    "business_impact": [18000, 200, 12000, 50]  # estimated $ lost if ignored
})
# Define thresholds
drift_threshold = 0.1  # only flag meaningful drift
impact_threshold = 1000  # only flag high business impact
# Filter actionable alerts
actionable = alerts[(alerts["drift_score"] > drift_threshold) &
                    (alerts["business_impact"] > impact_threshold)]
print("🔥 Actionable alerts this week:")
print(actionable)

🔥 Actionable alerts this week:
              feature  drift_score  business_impact
0      payment_status         0.35            18000
2  transaction_amount         0.25            12000
